### Notebook GAN para a Geração de Dados Pessoais Fictícios

In [10]:
import random
import string
import numpy as np
import pandas as pd
from tensorflow.keras import layers, Sequential
from tensorflow.keras.optimizers import Adam
from datetime import datetime, timedelta
from faker import Faker

fake = Faker('pt_BR')

def gerar_rg():
    """Função para gerar um número de RG brasileiro simulado"""
    rg_numero = ''.join([str(random.randint(0, 9)) for _ in range(8)])  
    digito = random.choice(string.digits + string.ascii_uppercase)  
    return rg_numero + digito  

def gerar_senha():
    """Função para gerar uma senha aleatória com letras, números e caracteres especiais"""
    tamanho_senha = random.randint(8, 12)  
    caracteres = string.ascii_letters + string.digits + string.punctuation
    senha = ''.join(random.choice(caracteres) for _ in range(tamanho_senha))
    return senha

def gerar_nome_email():
    """Função para gerar nome e e-mail de forma consistente"""
    nome = fake.name()  
    primeiro_nome = nome.split()[0].lower()  # Primeiro nome para o e-mail
    email = f"{primeiro_nome}@{fake.free_email_domain()}"
    return nome, email

def gera_dados_ficticios(n):
    """Função geradora de dados fictícios realistas com base no nome, CPF, RG, CNH e outros dados"""
    data = []  

    for _ in range(n):
        nome, email = gerar_nome_email()  # Gerar nome e e-mail juntos
        
        nascimento = fake.date_of_birth(minimum_age=18, maximum_age=80)  
        data_nascimento = nascimento.strftime('%d/%m/%Y')  
        
        ts = random.choice(["A-", "A+", "B-", "B+", "AB-", "AB+", "O-", "O+"])  
        
        cpf = fake.cpf().replace("-", "").replace(" ", "").replace(".", "")
        cpf = f"{cpf[:3]}.{cpf[3:6]}.{cpf[6:9]}-{cpf[9:]}"
        
        cnh = ''.join([str(random.randint(0, 9)) for _ in range(11)])  
        
        telefone = f"({random.randint(10, 99)}) {random.randint(90000, 99999)}-{random.randint(1000, 9999)}"  
        
        endereco = fake.address().replace("\n", ", ")  
        
        rg = gerar_rg()

        login = nome.split()[0].lower() + str(random.randint(100, 999))
        
        senha = gerar_senha()
        
        data.append([nome, data_nascimento, email, ts, cpf, rg, cnh, telefone, endereco, login, senha])

    return pd.DataFrame(data, columns=["Nome", "Data de Nascimento", "Email", "Tipo Sanguíneo", "CPF", "RG", "CNH", "Telefone", "Endereço", "Login", "Senha"])

# Função para gerar dados fictícios com o GAN (mas agora sem os campos de nome, email e CPF)
def build_generator(latent_dim):
    """Função para construir o gerador do modelo GAN"""
    model = Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dense(9, activation='sigmoid'))  
    return model

# Discriminador
def build_discriminator():
    """Função para construir o discriminador do modelo GAN"""
    model = Sequential()
    model.add(layers.Dense(512, activation='relu', input_dim=9))  
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  
    return model

# Modelo GAN
def build_gan(generator, discriminator):
    """Função para construir o modelo GAN combinando gerador e discriminador"""
    discriminator.trainable = False  
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

latent_dim = 100

generator = build_generator(latent_dim)
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

# Função para treinar o modelo GAN
def train_gan(epochs, batch_size, latent_dim, data):
    """Função para treinar o modelo GAN"""
    batch_count = data.shape[0] // batch_size
    for epoch in range(epochs):
        for _ in range(batch_count):
            # 1. Treinar o discriminador com dados reais e gerados
            real_data = data.sample(batch_size).values
            fake_data = gera_dados_ficticios(batch_size).values  # Gerando dados falsos com o gerador
            
            # Etiquetas para dados reais e gerados
            real_labels = np.ones((batch_size, 1))
            fake_labels = np.zeros((batch_size, 1))
            
            # Treinando o discriminador
            discriminator_loss_real = discriminator.train_on_batch(real_data, real_labels)
            discriminator_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
            
            # 2. Treinar o gerador
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            gan_labels = np.ones((batch_size, 1))  # O gerador quer enganar o discriminador
            
            generator_loss = gan.train_on_batch(noise, gan_labels)
        
        print(f"{epoch+1}/{epochs} | D Loss Real: {discriminator_loss_real[0]} | D Loss Fake: {discriminator_loss_fake[0]} | G Loss: {generator_loss[0]}")

# Gerar dados reais para treinamento
dados_ficticios = gera_dados_ficticios(1000)

# Gerar 100 novos registros com o gerador treinado (os campos de nome, email e CPF são fixos)
new_data = gera_dados_ficticios(100)

# Exportando para Excel
new_data.to_excel('dados_pessoais_ficticios_gerados3.xlsx', index=False)